# Batch

In [1]:
#import findspark
#findspark.init("/opt/spark-2.0.2-bin-hadoop2.7")
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import split, explode

import pyspark

import numpy

spark = (SparkSession.builder
    .master("local[*]")
    .config("spark.driver.cores", 1)
    .appName("understanding_sparksession")
    .getOrCreate() )
sc = spark.sparkContext
print(spark)
print(sc)

ImportError: No module named 'pyspark'

Carga del esquema, los datos y eliminamos NAs

In [ ]:
schema = StructType([
                        StructField("ID",StringType(),True),
                        StructField("PARENT-SYS-ID",StringType(),True),
                        StructField("Source",StringType(),True),
                        StructField("Mentions",StringType(),True),
                        StructField("Target",StringType(),True),
                        StructField("NAME Source",StringType(),True),
                        StructField("BODY",StringType(),True),
                        StructField("PUBDATE",TimestampType(),True),
                        StructField("URLs comma separated",StringType(),True),
                        StructField("Type TW-RT-MT",StringType(),True),
                        StructField("LINK",StringType(),True),
                        StructField("n1 Link",ByteType(),True),
                        StructField("n1 Picture",ByteType(),True),
                        StructField("PERSONAL-WEBSITE",StringType(),True),
                        StructField("COUNTRY",StringType(),True),
                        StructField("ALL-NICK-ACTIVITY-EVER",LongType(),True),
                        StructField("NICK-FOLLOWERS",LongType(),True),
                        StructField("FRIENDS-FOLLOWING-AUDIENCE",LongType(),True),
                        StructField("LOCATION",StringType(),True)
                   ])

#df = spark.read.csv("DATASET_Twitter-23-26-Mar-2014-MotoGP-Qatar.csv", 
#                    header=True, schema = schema, sep=",", timeStampFormat="dd/MM/yyyy HH:mm")

In [ ]:
df = (spark.read.format("csv").load("DATASET Twitter-23-26-Mar-2014-MotoGP-Qatar.csv", 
                    header=True, schema = schema, sep="\t", timeStampFormat="dd/MM/yyyy HH:mm"))

### 1. Contabilizar el número total de menciones a los pilotos...

#### Marc Márquez

In [ ]:
(df
 .filter(df.Mentions.rlike("marcmarquez93"))
 .count()
)

#### Valentino Rossi

In [ ]:
(df
 .filter(df.Mentions.rlike("valeyellow46"))
 .count()
)

#### Dani Pedrosa

In [ ]:
(df
 .filter(df.Mentions.rlike("26_danipedrosa"))
 .count()
)

###  2. Contabilizar los 5 países que más tweets han publicado (considerando los tweets que contengan dicha información).

In [ ]:
(df.filter(df.COUNTRY != "not public")
 .groupBy("COUNTRY")
 .count()
 .orderBy('count', ascending = False)
 .limit(5)
 .show())

### 3. Contabilizar los 3 hashtags más utilizados (que aparezcan el mayor número de veces) en el cuerpo de los tweets (campo "body").

In [ ]:
bodies = df.select(explode(split("BODY", "\s+")))

(bodies
 .filter(bodies.col.startswith("#"))
 .groupBy("col")
 .count()
 .orderBy('count', ascending = False)
 .limit(3)
 .show()
)

In [ ]:
#sc.stop()